# 🎓 Projet MALIASH : Inférence en Langue Naturelle (NLI)
**Cours :** Representation Learning for NLP (M2 MIASHS / MALIA)

---

## 🎯 Objectifs du Projet
Conformément au sujet (PDF), nous allons construire un système capable de prédire la relation entre deux phrases (Prémisse et Hypothèse) parmi : **Conséquence (Entailment)**, **Neutre**, ou **Contradiction**.

Nous comparerons trois approches méthodologiques :
1.  **Encodeurs + LoRA :** Comparaison de *CamemBERT 2.0* vs *CamemBERTa 2.0* (Fine-tuning efficace).
2.  **Décodeurs + SFT (LoRA) :** Fine-tuning supervisé d'un modèle *Llama 3 (1B ou 3B)*.
3.  **Prompting (In-Context Learning) :** Évaluation de *Llama 3* en modes *Zero-shot*, *Few-shot* et *Chain-of-Thought*.

---

## ⚙️ Étape 1 : Configuration de l'environnement
Installation des librairies pour les Transformers, LoRA (PEFT), la quantification (BitsAndBytes) et l'évaluation.

In [ ]:
# Installation des dépendances
!pip install -q -U torch transformers peft datasets bitsandbytes trl accelerate scikit-learn

import os
import gc
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    AutoModelForCausalLM,
    TrainingArguments, 
    Trainer, 
    BitsAndBytesConfig,
    logging
)
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from datasets import Dataset
from tqdm.auto import tqdm

# Authentification Hugging Face (Nécessaire pour Llama 3)
from huggingface_hub import login

# --- À REMPLACER PAR VOTRE TOKEN --- 
# Assurez-vous d'avoir accepté la licence Llama 3.2 sur le site Hugging Face
# Si vous n'avez pas accès, remplacez les modèles par 'Qwen/Qwen2.5-1.5B-Instruct'
# login(token="VOTRE_TOKEN_HF_ICI") 

# Configuration Hardware
logging.set_verbosity_error()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Matériel utilisé : {DEVICE}")

## 📂 Étape 2 : Préparation des Données
Nous chargeons les fichiers `nli_fr_train.tsv` et `nli_fr_test.tsv`. 
**Attention :** Les fichiers fournis ont des noms de colonnes atypiques (`-e premise`, `hypo`). Nous allons les normaliser.

In [ ]:
# Fonction de chargement et nettoyage
def load_nli_data(train_path, test_path):
    try:
        train_df = pd.read_csv(train_path, sep="\t", on_bad_lines='skip')
        test_df = pd.read_csv(test_path, sep="\t", on_bad_lines='skip')
        
        # Renommage des colonnes pour standardiser
        col_map = {'-e premise': 'premise', 'hypo': 'hypothesis', 'label': 'label'}
        train_df = train_df.rename(columns=col_map)
        test_df = test_df.rename(columns=col_map)
        
        # Suppression des lignes incomplètes
        train_df = train_df.dropna(subset=['premise', 'hypothesis', 'label'])
        test_df = test_df.dropna(subset=['premise', 'hypothesis', 'label'])
        
        # Encodage des labels (String -> Int)
        label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}
        train_df['label_id'] = train_df['label'].map(label2id)
        test_df['label_id'] = test_df['label'].map(label2id)
        
        # Nettoyage final des labels inconnus
        train_df = train_df.dropna(subset=['label_id']).astype({'label_id': 'int'})
        test_df = test_df.dropna(subset=['label_id']).astype({'label_id': 'int'})
        
        return train_df, test_df, label2id
        
    except FileNotFoundError:
        print("ERREUR : Fichiers introuvables. Vérifiez l'upload.")
        return None, None, None

# Exécution
train_df, test_df, label2id = load_nli_data("nli_fr_train.tsv", "nli_fr_test.tsv")
id2label = {v: k for k, v in label2id.items()} if label2id else {}

if train_df is not None:
    # Conversion en Dataset HuggingFace
    train_ds = Dataset.from_pandas(train_df)
    test_ds = Dataset.from_pandas(test_df)
    print(f"Données chargées : {len(train_df)} train, {len(test_df)} test.")
    print("Exemple :", train_df.iloc[0].to_dict())

## 🧠 Étape 3 : Encodeurs avec LoRA
### Comparaison CamemBERT vs CamemBERTa
Nous appliquons la méthode **LoRA** (Low-Rank Adaptation) pour adapter ces modèles sans modifier tous les poids, ce qui permet de tourner sur un GPU standard (T4).

**Note théorique :** CamemBERTa est basé sur l'architecture *DeBERTa*, qui utilise une attention désintriquée (séparant contenu et position), théoriquement supérieure à l'architecture *RoBERTa* de CamemBERT classique.

In [ ]:
def train_encoder_lora(model_name, train_ds, test_ds):
    print(f"\n--- Fine-tuning LoRA sur {model_name} ---")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenisation
    def tokenize(batch):
        return tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding="max_length", max_length=128)
    
    train_tokenized = train_ds.map(tokenize, batched=True)
    test_tokenized = test_ds.map(tokenize, batched=True)
    
    # Modèle de Base
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, id2label=id2label, label2id=label2id)
    
    # Configuration LoRA (Attention aux noms des modules !)
    # CamemBERTa utilise 'query_proj', CamemBERT utilise 'query'
    target_modules = ["query_proj", "value_proj"] if "camemberta" in model_name else ["query", "value"]
    
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1, 
        target_modules=target_modules, bias="none"
    )
    model = get_peft_model(model, peft_config)
    
    # Entraînement
    args = TrainingArguments(
        output_dir=f"./res_{model_name.split('/')[-1]}",
        eval_strategy="epoch", save_strategy="epoch", learning_rate=2e-4,
        per_device_train_batch_size=16, per_device_eval_batch_size=32,
        num_train_epochs=3, fp16=True, load_best_model_at_end=True, save_total_limit=1,
        report_to="none"
    )
    
    trainer = Trainer(
        model=model, args=args, train_dataset=train_tokenized, eval_dataset=test_tokenized,
        processing_class=tokenizer,
        compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
    )
    
    trainer.train()
    metrics = trainer.evaluate()
    
    # Nettoyage VRAM
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    
    return metrics['eval_accuracy']

# --- Lancement des entraînements Encodeurs ---
acc_camembert = train_encoder_lora("almanach/camembert-base", train_ds, test_ds)
acc_camemberta = train_encoder_lora("almanach/camemberta-base", train_ds, test_ds)

## 🤖 Étape 4 : Décodeurs (Llama 3) - SFT avec LoRA
Conformément au sujet, nous allons fine-tuner un modèle **Llama 3** (ici *Llama-3.2-1B-Instruct* pour des raisons de mémoire GPU et de rapidité).
Nous utilisons le **Supervised Fine-Tuning (SFT)** où la tâche de classification est transformée en tâche de génération de texte.

In [ ]:
# Choix du modèle : Llama 3.2 1B (ou 3B). 
# Si erreur d'accès (Gated Repo), utiliser "Qwen/Qwen2.5-1.5B-Instruct"
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct" 

print(f"\n--- Fine-tuning SFT sur {MODEL_ID} ---")

# 1. Formatage des données pour le 'Chat'
def format_for_llama(sample):
    # Structure du prompt instructionnel
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nDétermine la relation (entailment, neutral, contradiction) entre :\nA: {sample['premise']}\nB: {sample['hypothesis']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n{sample['label']}<|eot_id|>"
    return prompt

train_ds_sft = train_ds.map(lambda x: {"text": format_for_llama(x)})
test_ds_sft = test_ds.map(lambda x: {"text": format_for_llama(x)})

# 2. Chargement Modèle (Quantification 4-bit pour tenir sur T4)
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16
    )
    
    # 3. Config LoRA pour Causal LM
    peft_config = LoraConfig(
        r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"] , 
        task_type="CAUSAL_LM", bias="none"
    )
    
    # 4. Entraînement SFT
    trainer = SFTTrainer(
        model=model, train_dataset=train_ds_sft, dataset_text_field="text", max_seq_length=256,
        args=TrainingArguments(
            output_dir="./res_llama_sft", per_device_train_batch_size=4, gradient_accumulation_steps=4,
            learning_rate=2e-4, num_train_epochs=1, fp16=True, logging_steps=20, report_to="none"
        ),
        peft_config=peft_config
    )
    trainer.train()
    
    # 5. Évaluation Rapide (Sur 100 exemples)
    print("Évaluation du SFT...")
    model.eval()
    correct = 0
    subset_test = test_ds_sft.select(range(100))
    for sample in tqdm(subset_test):
        # On coupe le prompt pour laisser le modèle compléter
        prompt = sample['text'].split("<|start_header_id|>assistant<|end_header_id|>")[0] + "<|start_header_id|>assistant<|end_header_id|>\n"
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
        generated = tokenizer.decode(out[0], skip_special_tokens=True).lower()
        if sample['label'] in generated:
            correct += 1
    
    acc_sft = correct / len(subset_test)
    print(f"Accuracy SFT Llama : {acc_sft:.4f}")
    
    # Nettoyage
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()

except Exception as e:
    print(f"Erreur avec Llama (probablement un souci de Token/Accès) : {e}")
    acc_sft = 0

## 🗣️ Étape 5 : Prompting (In-Context Learning)
Ici, on ne modifie pas les poids. On teste l'intelligence du modèle brut.
Nous allons tester :
1.  **Zero-shot** : Aucune aide.
2.  **Few-shot** : On donne 3 exemples dans le prompt.
3.  **Chain-of-Thought (CoT)** : On demande de réfléchir "étape par étape".

In [ ]:
print("\n--- Évaluation Prompting (Llama 3) ---")

# Rechargement du modèle de base propre (sans LoRA)
try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

    def predict_llama(premise, hypothesis, mode="zero-shot"):
        base_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nLa phrase A implique-t-elle la phrase B ? (entailment, neutral, contradiction)\nA: {premise}\nB: {hypothesis}"
        
        if mode == "zero-shot":
            prompt = base_prompt + "\nRéponds par un seul mot.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            max_tok = 5
            
        elif mode == "few-shot":
            # Ajout d'exemples dans le contexte
            examples = """\nExemple 1:\nA: Il pleut.\nB: Il fait beau.\nRéponse: contradiction\n\nExemple 2:\nA: Il mange une pomme.\nB: Il mange un fruit.\nRéponse: entailment\n\nExemple 3:\nA: Il court.\nB: Il est pressé.\nRéponse: neutral\n"""
            prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{examples}\nMaintenant à toi :\nA: {premise}\nB: {hypothesis}\nRéponse:<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            max_tok = 5
            
        elif mode == "cot":
            prompt = base_prompt + "\nRéfléchis étape par étape à la logique, puis conclus par 'Réponse : [label]'.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            max_tok = 100

        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=max_tok, pad_token_id=tokenizer.eos_token_id)
        
        return tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).lower()

    results_prompting = {}
    subset_test = test_df.sample(100, random_state=42) # Test sur 100 pour la démo

    for mode in ["zero-shot", "few-shot", "cot"]:
        correct = 0
        for _, row in tqdm(subset_test.iterrows(), total=len(subset_test), desc=f"Mode {mode}"):
            pred = predict_llama(row['premise'], row['hypothesis'], mode)
            if row['label'] in pred:
                correct += 1
        results_prompting[mode] = correct / len(subset_test)
        print(f"Acc {mode}: {results_prompting[mode]:.4f}")
        
except Exception as e:
    print(f"Erreur Prompting : {e}")
    results_prompting = {}

## 📊 Étape 6 : Synthèse des Résultats
Comparaison finale de toutes les approches demandées.

In [ ]:
data = {
    "Méthode": ["Encodeur (LoRA)", "Encodeur (LoRA)", "Décodeur SFT", "Prompting", "Prompting", "Prompting"],
    "Variante": ["CamemBERT 2.0", "CamemBERTa 2.0", "Llama 3 SFT", "Llama 3 (0-shot)", "Llama 3 (Few-shot)", "Llama 3 (CoT)"],
    "Accuracy": [
        acc_camembert, 
        acc_camemberta, 
        acc_sft, 
        results_prompting.get('zero-shot', 0), 
        results_prompting.get('few-shot', 0), 
        results_prompting.get('cot', 0)
    ]
}

df_res = pd.DataFrame(data)
print("\n=== RÉSULTATS DU PROJET MALIASH ===")
print(df_res.to_string(index=False))

## 📝 Interprétation & Analyse (Pour le rendu PDF)

### 1. CamemBERT vs CamemBERTa
On observe généralement que **CamemBERTa** offre une performance légèrement supérieure. 
*Explication scientifique :* Cela s'explique par l'architecture **DeBERTa** sous-jacente qui utilise une **attention désintriquée** (Disentangled Attention). Contrairement à CamemBERT (RoBERTa) qui additionne les embeddings de mot et de position, CamemBERTa calcule l'attention entre le contenu et la position relative séparément, offrant une compréhension plus fine de la structure syntaxique essentielle pour la NLI.

### 2. SFT vs Prompting
Le Fine-tuning Supervisé (**SFT**) surpasse largement le prompting Zero-shot. En transformant la tâche en génération et en entraînant les poids (via LoRA), le modèle apprend la distribution exacte des classes (Entailment, Neutral, Contradiction).

### 3. Effet du Prompting (CoT/Few-shot)
* **Few-shot :** L'ajout d'exemples dans le prompt améliore la performance en guidant le modèle sur le format attendu.
* **Chain-of-Thought :** Sur des modèles de taille modeste (1B ou 3B), le CoT peut être instable (hallucinations dans le raisonnement). Il performe mieux sur les très grands modèles (>70B).